In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [69]:
def check_df(df):
    #print(df.head())
    for i, col_name in enumerate(df):
        print(i, col_name, df[col_name].dtypes)
        if df[col_name].hasnans:
            print(df[col_name].isna().value_counts())
        print('')
        if len(df[col_name].unique())<10:
            print(df[col_name].value_counts())
        print('')

In [70]:
file = '전국평생학습강좌표준데이터 (1).csv'
pd_data = pd.read_csv(file, encoding = 'cp949')
check_df(pd_data)

0 강좌명 object


1 강사명 object


2 교육시작일자 object


3 교육종료일자 object


4 교육시작시각 object
False    22167
True      4943
Name: 교육시작시각, dtype: int64


5 교육종료시각 object
False    22167
True      4943
Name: 교육종료시각, dtype: int64


6 강좌내용 object


7 교육대상구분 object


8 교육방법구분 object

오프라인             21616
온라인               3852
혼합                1090
방문교육               345
오프라인+온라인            97
온라인+오프라인            64
오프라인+방문교육           43
온라인+모바일              2
오프라인+모바일+우편통신        1
Name: 교육방법구분, dtype: int64

9 운영요일 object
False    24500
True      2610
Name: 운영요일, dtype: int64


10 교육장소 object
False    25095
True      2015
Name: 교육장소, dtype: int64


11 강좌정원수 int64


12 수강료 int64


13 교육장도로명주소 object


14 운영기관명 object


15 운영기관전화번호 object


16 접수시작일자 object
False    21672
True      5438
Name: 접수시작일자, dtype: int64


17 접수종료일자 object
False    21663
True      5447
Name: 접수종료일자, dtype: int64


18 접수방법구분 object


19 선정방법구분 object
False    23391
True      3719
Name: 선정방법구분, dtype: int64


20 홈페이지주소 object

In [71]:
#pd_data['운영기관명'].unique()
#pd_data['제공기관명'].unique()


# 전처리 과정
def area_from_address(full_address_str):
    if '[' in full_address_str:
        i = full_address_str.find(']')+1
        full_address_str = full_address_str[i:]
        
    if '?' in full_address_str:
        full_address_str = full_address_str.split('?')[0]
        
    return full_address_str.split()[0]

def rewrite(area_name):
    short_name = ['경남', '경북', '전남', '전북', '부산', '광주', '서울', '인천']
    long_name = ['경상남도', '경상북도', '전라남도', '전라북도', '부산광역시', '광주광역시', '서울특별시', '인천광역시']
    
    if area_name in short_name:
        i = short_name.index(area_name)
        return long_name[i]
    else:
        return area_name

def rewrite2(area_name):
    if area_name == '마산합포구':
        return '경상남도'
    elif area_name == '고흥군':
        return '전라남도'
    elif area_name == '서울특별시중랑구':
        return '서울특별시'
    else:
        return area_name


area_names = pd_data['교육장도로명주소'].apply(area_from_address)
area_names = area_names.apply(rewrite)
area_names = area_names.apply(rewrite2)
#area_names.unique()
cnt = area_names.value_counts()
ignore_names = list(cnt['.':].index)

for i,n in enumerate(area_names):
    if n in ignore_names:
        area_names[i] = '몰라'
        
print(area_names.unique())
pd_data['area_names'] = area_names # 원래 pd_data의 마지막 column에 추가

['경기도' '서울특별시' '강원도' '경상남도' '몰라' '경상북도' '충청북도' '부산광역시' '대구광역시' '울산광역시'
 '광주광역시' '충청남도' '대전광역시' '전라북도' '전라남도' '인천광역시' '세종특별자치시' '당진시' '제주특별자치도']


In [72]:
#pd_data.head()
pd_data['area_names'].value_counts()

경기도        8149
경상남도       2767
광주광역시      2214
서울특별시      1969
전라북도       1631
대구광역시      1627
인천광역시      1395
경상북도       1347
강원도         916
전라남도        887
부산광역시       844
충청남도        645
대전광역시       534
제주특별자치도     467
충청북도        446
몰라          427
당진시         403
울산광역시       296
세종특별자치시     146
Name: area_names, dtype: int64

In [73]:
delivery_types = pd_data['교육방법구분']
delivery_types.unique()

def rewrite3(type_name):
    if '+' in type_name:
        return '혼합'
    else:
        return type_name


#delivery_types = delivery_types.apply(rewrite3)
# lambda 식 ==> 간단한 조건이여서 함수를 구현할 정도는 아닐 때 사용한다. 
delivery_types = delivery_types.apply(lambda type_name: '혼합' if '+' in type_name else type_name)
print(delivery_types.unique())
print(delivery_types.value_counts())

pd_data['delivery_types'] = delivery_types # == pd_data.insert(28, 'delivery_types', delivery_types)

['혼합' '오프라인' '온라인' '방문교육']
오프라인    21616
온라인      3852
혼합       1297
방문교육      345
Name: 교육방법구분, dtype: int64


In [74]:
pd_data.iloc[:, -5:].head()

,데이터기준일자,제공기관코드,제공기관명,area_names,delivery_types
0,2021-09-06,4080000,경기도 안성시,경기도,혼합
1,2021-09-06,4080000,경기도 안성시,경기도,혼합
2,2021-09-06,4080000,경기도 안성시,경기도,혼합
3,2021-09-06,4080000,경기도 안성시,경기도,혼합
4,2021-09-06,4080000,경기도 안성시,경기도,혼합


In [75]:
sub_df = pd_data.iloc[:, -2:]
sub_df

,area_names,delivery_types
0,경기도,혼합
1,경기도,혼합
2,경기도,혼합
3,경기도,혼합
4,경기도,혼합
...,...,...
27105,경상북도,오프라인
27106,경상북도,오프라인
27107,경상북도,오프라인
27108,경상북도,오프라인


In [80]:
for area_name in sub_df['area_names'].unique():
    print(area_name)
    filter1 = sub_df['area_names'] == area_name
    area_df = sub_df[filter1]
    print(area_df['delivery_types'].value_counts())
    print('')

경기도
오프라인    6509
온라인      792
혼합       652
방문교육     196
Name: delivery_types, dtype: int64

서울특별시
오프라인    1379
온라인      524
혼합        66
Name: delivery_types, dtype: int64

강원도
오프라인    738
온라인     170
혼합        8
Name: delivery_types, dtype: int64

경상남도
오프라인    2695
온라인       72
Name: delivery_types, dtype: int64

몰라
오프라인    369
온라인      57
혼합        1
Name: delivery_types, dtype: int64

경상북도
오프라인    1036
온라인      309
혼합         1
방문교육       1
Name: delivery_types, dtype: int64

충청북도
오프라인    298
혼합      130
온라인      18
Name: delivery_types, dtype: int64

부산광역시
오프라인    724
온라인      57
방문교육     45
혼합       18
Name: delivery_types, dtype: int64

대구광역시
오프라인    1591
온라인       31
혼합         5
Name: delivery_types, dtype: int64

울산광역시
오프라인    267
온라인      28
방문교육      1
Name: delivery_types, dtype: int64

광주광역시
오프라인    1805
온라인      315
혼합        94
Name: delivery_types, dtype: int64

충청남도
오프라인    574
온라인      35
방문교육     33
혼합        3
Name: delivery_types, dtype: int64

대전광역시
오프라인    336
혼합

In [88]:
# groupby
sub_df = pd_data[['delivery_types', 'area_names', '수강료']]
g1 = sub_df.groupby(['area_names', 'delivery_types'])
g1.count()
df10 = g1.agg([np.mean, np.max])    # 밑에 표에서 index는 area_names, delivery_types이다 즉, 멀티 인덱스이다.
                                    # 밑에 표에서 mean, amax도 멀티 인덱스이다.
df10

수강료        
                                   mean    amax
area_names delivery_types                      
강원도        오프라인            13726.626016  800000
           온라인             31235.294118   80000
           혼합              11250.000000   50000
경기도        방문교육            67755.102041  100000
           오프라인            31995.324935  400000
...                                 ...     ...
충청남도       온라인                 0.000000       0
           혼합                  0.000000       0
충청북도       오프라인            18775.167785   40000
           온라인              6666.666667   40000
           혼합              27153.846154   50000

[61 rows x 2 columns]

In [96]:
#print(df10.index)
#print(df10.columns)
#df10[('수강료', 'mean')]
#print(df10)
print(df10.loc[('강원도', '오프라인'), :]) # 결과값은 Series로 반환해준다.

수강료  mean     13726.626016
     amax    800000.000000
Name: (강원도, 오프라인), dtype: float64
